In [41]:
## import library 
import os
import pandas as pd
import numpy as np

#import google cloud library
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform

import pickle
from xgboost import XGBClassifier

In [42]:
new_test_data = pd.DataFrame({
    'Dependents':['Yes','No'],
    'Tenure':[61,18],
    'OnlineSecurity':['No internet service','No'], 
    'OnlineBackup':['No internet service','No'],
    'InternetService':['No','Fiber optic'], 
    'DeviceProtection':['No internet service','No'], 
    'TechSupport':['No internet service','No'], 
    'Contract':['Two year','Month-to-month'],
    'PaperlessBilling':['Yes','No'], 
    'MonthlyCharges':[25.0,78.55]
})

new_test_data

,Dependents,Tenure,OnlineSecurity,OnlineBackup,InternetService,DeviceProtection,TechSupport,Contract,PaperlessBilling,MonthlyCharges
0,Yes,61,No internet service,No internet service,No,No internet service,No internet service,Two year,Yes,25.00
1,No,18,No,No,Fiber optic,No,No,Month-to-month,No,78.55


In [43]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../randy-029/sa-development.json"

project_id = 'dti-ds'
dataset_id = 'randy_dataset_029'
table_id = 'telco_churn_real'
region = 'us-central1'
bucket_name = 'randy_gcs_029'
blob_name = 'data/telco_churn_real.csv'

In [44]:
# Upload the model to Google Cloud Storage
try : 
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name) # Add bucket name
    blob_model = bucket.blob('model/best_capstone_model_xgb.sav')
    blob_model.upload_from_filename('best_capstone_model_xgb.sav')

    print ("Uploading model succeeded")
except:
    raise TypeError("An exception occurred")

Uploading model succeeded


In [45]:
#load data from BQ
## using bigquery client 
# client = bigquery.Client(credentials=credentials,project=project_id)
client = bigquery.Client(project=project_id)

# query 
query_job = client.query(f"""select * from {dataset_id}.{table_id}""")
auto_cloud = query_job.result().to_dataframe()

auto_cloud.columns = new_test_data.columns
#cleansing 
# result = df.drop(['ID'], axis = 1)
auto_cloud

/home/randy_prawiro/miniconda3/envs/telco-churn-env/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,Dependents,Tenure,OnlineSecurity,OnlineBackup,InternetService,DeviceProtection,TechSupport,Contract,PaperlessBilling,MonthlyCharges
0,True,49,No,Yes,DSL,No,No,One year,True,51.80
1,False,50,No,Yes,Fiber optic,No,No,One year,True,98.25
2,True,58,No,No,DSL,No,No,One year,True,44.10
3,False,11,No,Yes,DSL,No,No,One year,True,61.25
4,True,23,Yes,No,Fiber optic,No,No,One year,False,81.85
...,...,...,...,...,...,...,...,...,...,...
966,False,2,No internet service,No internet service,No,No internet service,No internet service,Month-to-month,False,20.65
967,False,1,No internet service,No internet service,No,No internet service,No internet service,Month-to-month,False,19.75
968,False,1,No internet service,No internet service,No,No internet service,No internet service,Month-to-month,False,21.10
969,False,11,No internet service,No internet service,No,No internet service,No internet service,Month-to-month,False,19.95


In [46]:
auto_cloud['Dependents'] = auto_cloud['Dependents'].astype(str).replace({'True':'Yes', 'False':'No'})
auto_cloud['PaperlessBilling'] = auto_cloud['PaperlessBilling'].astype(str).replace({'True':'Yes', 'False':'No'})
auto_cloud

,Dependents,Tenure,OnlineSecurity,OnlineBackup,InternetService,DeviceProtection,TechSupport,Contract,PaperlessBilling,MonthlyCharges
0,Yes,49,No,Yes,DSL,No,No,One year,Yes,51.80
1,No,50,No,Yes,Fiber optic,No,No,One year,Yes,98.25
2,Yes,58,No,No,DSL,No,No,One year,Yes,44.10
3,No,11,No,Yes,DSL,No,No,One year,Yes,61.25
4,Yes,23,Yes,No,Fiber optic,No,No,One year,No,81.85
...,...,...,...,...,...,...,...,...,...,...
966,No,2,No internet service,No internet service,No,No internet service,No internet service,Month-to-month,No,20.65
967,No,1,No internet service,No internet service,No,No internet service,No internet service,Month-to-month,No,19.75
968,No,1,No internet service,No internet service,No,No internet service,No internet service,Month-to-month,No,21.10
969,No,11,No internet service,No internet service,No,No internet service,No internet service,Month-to-month,No,19.95


In [47]:
# Load the model
with open('best_capstone_model_xgb.sav', 'rb') as model:
    loaded_model = pickle.load(model)

y_pred_cloud_new_data = loaded_model.predict(new_test_data)
y_pred_cloud_new_data

array([0, 1])

In [48]:
new_test_cloud = new_test_data.copy()
new_test_cloud['Churn Prediction'] = y_pred_cloud_new_data # prediction column
new_test_cloud

,Dependents,Tenure,OnlineSecurity,OnlineBackup,InternetService,DeviceProtection,TechSupport,Contract,PaperlessBilling,MonthlyCharges,Churn Prediction
0,Yes,61,No internet service,No internet service,No,No internet service,No internet service,Two year,Yes,25.00,0
1,No,18,No,No,Fiber optic,No,No,Month-to-month,No,78.55,1


In [49]:
y_pred_file_cloud = loaded_model.predict(auto_cloud)
y_pred_file_cloud[:13]

array([0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])

In [50]:
auto_cloud['Churn Prediction'] = y_pred_file_cloud # prediction column
auto_cloud

,Dependents,Tenure,OnlineSecurity,OnlineBackup,InternetService,DeviceProtection,TechSupport,Contract,PaperlessBilling,MonthlyCharges,Churn Prediction
0,Yes,49,No,Yes,DSL,No,No,One year,Yes,51.80,0
1,No,50,No,Yes,Fiber optic,No,No,One year,Yes,98.25,1
2,Yes,58,No,No,DSL,No,No,One year,Yes,44.10,0
3,No,11,No,Yes,DSL,No,No,One year,Yes,61.25,0
4,Yes,23,Yes,No,Fiber optic,No,No,One year,No,81.85,0
...,...,...,...,...,...,...,...,...,...,...,...
966,No,2,No internet service,No internet service,No,No internet service,No internet service,Month-to-month,No,20.65,1
967,No,1,No internet service,No internet service,No,No internet service,No internet service,Month-to-month,No,19.75,1
968,No,1,No internet service,No internet service,No,No internet service,No internet service,Month-to-month,No,21.10,1
969,No,11,No internet service,No internet service,No,No internet service,No internet service,Month-to-month,No,19.95,0


In [51]:
table_id = 'telco_churn_prediction'

# Construct a BigQuery client object
client = bigquery.Client()

# Define the full table ID
table_full_id = f"{client.project}.{dataset_id}.{table_id}"

auto_cloud.columns = ['_'.join(i.split(' ')) for i in auto_cloud.columns] # no empty spaces in column names

# Data formatting
auto_cloud['Dependents'] = auto_cloud['Dependents'].replace({'Yes':True, 'No':False})
auto_cloud['PaperlessBilling'] = auto_cloud['PaperlessBilling'].replace({'Yes':True, 'No':False})

# Load the DataFrame into the BigQuery table
job = client.load_table_from_dataframe(auto_cloud, table_full_id)

# Wait for the job to complete
job.result()
print(f"Loaded {job.output_rows} rows into {table_full_id}")

/tmp/ipykernel_8246/2407127859.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  auto_cloud['Dependents'] = auto_cloud['Dependents'].replace({'Yes':True, 'No':False})
/tmp/ipykernel_8246/2407127859.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  auto_cloud['PaperlessBilling'] = auto_cloud['PaperlessBilling'].replace({'Yes':True, 'No':False})


Loaded 971 rows into dti-ds.randy_dataset_029.telco_churn_prediction
